# A simple network to classify handwritten digits


Trong lab 3, chúng ta sẽ xây dựng những neural network đơn giản để phân loại các chữ số viết tay trong kho dữ liệu MNIST.

Phần 1: neural network này sẽ chỉ có input layer và output layer. Ta sẽ sử dụng softmax regression cho output layer, và hàm cross entropy khi tính loss function. Với cách chọn hàm như vậy, network này hoàn toàn tương đồng với thuật toán logistic regression cho nhiều loại trong lab trước.

Phần 2: bài tập thêm vào neural network ở phần 1 một hidden layer ở giữa input và output layer.

Phần 3: bài tập thêm hidden layer nữa vào network ở phần 2.

Slide đi kèm: https://cloud.google.com/blog/big-data/2017/01/learn-tensorflow-and-deep-learning-without-a-phd.


## 1. Network với input và output layer
### 1.1 Khái quát
(Dựa theo https://www.tensorflow.org/get_started/mnist/beginners)
Trong phần này, chúng ta sẽ xây dựng một network không có hidden layer mà chỉ có input layer gồm 784 neuron được kết nối thẳng đến output layer gồm 10 neuron tương ứng với 10 nhóm chữ số viết tay.

<img src="../../images/graphs/neural_network_0_hidden_layer.png"/>

Kết quả của 10 neuron này sẽ được biến đổi ở bước softmax regression để cuối cùng ta thu được 10 giá trị xác suất (là các số thực dương và tổng là 1) ứng với xác suất mỗi hình ảnh thuộc về từng nhóm chữ số. Cuối quá trình học, hình ảnh sẽ được network gắn cho nhóm chữ số mà xác suất hình ảnh này ứng với nhóm đó là cao nhất trong 10 giá trị xác suất.

Sai số giữa label dự đoán và label thật được tính bằng hàm cross entropy. Một lần nữa chúng ta sẽ dùng $GradientDescentOptimizer$ để thay đổi $weight$ và $bias$ giữa input layer và output layer, sao cho sai số này là nhỏ nhất.

### 1.2 Xây dựng network
Network đơn giản này được xây dựng tương tự như Logistic Regression cho 10 nhóm trong lab trước. Chú ý cách sử dụng $Dataset.next\_batch()$ để lấy một nhóm nhỏ data đưa vào $train\_step$. Theo bạn tại sao chúng ta làm như vậy?

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

with tf.name_scope("Input") as scope:
    x = tf.placeholder(tf.float32, [None, 784], name="x-input")               
    y_correct = tf.placeholder(tf.float32, [None, 10], name="y-correct_label")
    
with tf.name_scope("Weight") as scope:
    W = tf.Variable(tf.zeros([784, 10]), name="weight")                       
    
with tf.name_scope("Bias") as scope:
    b = tf.Variable(tf.zeros(10), name="bias")                                
    
with tf.name_scope("Softmax") as scope:
    y = tf.nn.softmax(tf.matmul(x, W) + b, name="softmax")                      # * 
    
with tf.name_scope("Cross_Entropy") as scope:                                   # *
    cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_correct * tf.log(y), reduction_indices=[1]), name="cross_entropy")
    
with tf.name_scope('Train') as scope:
    train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)
    
with tf.name_scope("Accuracy"):
    correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_correct,1), name = "correct_prediction")
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

# Launch the model
sess = tf.InteractiveSession()
file_writer = tf.summary.FileWriter("DNN_0_hidden_layer", sess.graph)
# create a summary for our cost and accuracy
tf.summary.scalar("cost_summary", cross_entropy)
tf.summary.scalar("accuracy", accuracy)

# merge all summaries into a single operation which we can execute in a session 
summary_step = tf.summary.merge_all()

tf.global_variables_initializer().run()

for i in range(1000):
    batch_xs, batch_ys = mnist.train.next_batch(100)                         #batch_size = 100
    _, summary = sess.run([train_step, summary_step], feed_dict={x: batch_xs,y_correct: batch_ys})
    
    # logging
    file_writer.add_summary(summary, i)

\*: 2 bước này được viết tách biệt để thấy rõ trình tự các bước. Khi viết code bằng TensorFlow nên sử dụng $tf.nn.softmax\_cross\_entropy\_with\_logits$ trực tiếp lên $tf.matmul(x, W) + b$ như giải thích trong https://www.tensorflow.org/get_started/mnist/beginners#training.

## 2. Network với 1 hidden layer
### 2.1 Khái quát
<img src="../../images/graphs/neural_network_1_hidden_layer.png"/>

Chúng ta sẽ thêm vào network đơn giản ở trên 1 hidden layer nữa. Đầu tiên hãy chọn số neuron cho lớp ẩn này - một số nguyên bất kì $neuron\_1$, chúng ta sẽ thay đổi số nguyên này sau khi xây dựng model hoàn chỉnh.

<b>Trả lời:</b> $neuron\_1$ = ?

#### Input layer đến hidden layer
##### Inputs
Hãy mô tả shape của các vector sau:
* input $X$
* weight $W\_1$
* bias $b\_1$
* weighted sum $z$
* hidden layer result $a$

##### Activation function
Chọn một trong các activation function sau đây: linear, sigmoid ($tf.sigmoid$), hyperbolic tangent, ReLU ($tf.nn.relu$).

<b>Trả lời:</b> Activation function?


#### Hidden layer đến output layer
##### Inputs
Hãy mô tả shape của các vector sau:
* hidden layer result $a$
* weight $W\_2$
* bias $b\_2$
* weighted sum $z$
* output $y$

##### Activation function
Activation function ở output layer có thể là bất kì activation function nào liệt kê ở trên, tuy nhiên softmax thường được sử dụng. 

<i>Lưu ý cách dùng $tf.nn.softmax\_cross\_entropy\_with\_logits$ trong TensorFlow.</i> 

#### Cost/Loss Function, Optimizer

Cross entropy loss function

$H_{y'}(y) = -\sum_i y'_i \log(y_i)$

TensorFlow cung cấp sẵn một số optimizers, $GradientDescentOptimizer$ chỉ là một trong số đó: https://www.tensorflow.org/api_guides/python/train#Optimizers.

Chọn một trong các optimizers sau: GradientDescentOptimizer, AdamOptimizer, AdadeltaOptimizer.

<b>Trả lời:</b> Optimizer?



### 2.2 Xây dựng network

### Load data
Đầu tiên, import các thư viện cần thiết và dữ liệu MNIST như các lab trước.

In [ ]:
# Import tensorflow, matplotlib, load MNIST dataset (one_hot=True)
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

### Các tham số trong training

In [ ]:
# Training params
neuron_1 = 
num_iterations = 
batch_size = 

### Input
Khởi tạo tensor $x$ và $y\_correct$, chứa ảnh và label đúng của dataset.

In [ ]:
with tf.name_scope("Input") as scope:
    x = 
    y_correct =

### Input layer -> Hidden layer
Khởi tạo weight $W\_1$, bias $b\_1$, và tính weighted sum $z\_1$, hidden layer output $a$. Lưu ý khi khởi tạo weight và bias, ta có thể dùng $tf.random\_normal(shape)$ thay vì $tf.zeros(shape)$ để các giá trị ban đầu của các tham số là khác nhau. Xem thêm https://www.tensorflow.org/api_docs/python/tf/random_normal.

In [ ]:
with tf.name_scope("Input_to_Hidden_layer") as scope:
    W_1 = 
    b_1 = 
    z_1 = 
    a = 

### Hidden layer -> Output layer
Khởi tạo weight $W\_2$, bias $b\_2$, và tính label dự đoán $y$. 

In [ ]:
with tf.name_scope("Hidden_Layer_to_Output") as scope:
    W_2 = 
    b_2 = 
    y = 

### Optimization
Hãy định nghĩa hàm sai số cost/loss function dựa trên label thật $y\_correct$ và label dự đoán $y$. Chú ý xem $y$ đã được normalized chưa ở bước trên.

In [ ]:
with tf.name_scope("Cost_function") as scope:
    cost =

Chọn một optimizer để điều chỉnh thông số trong network sao cho $cost$ ở trên giảm đi.

In [ ]:
with tf.name_scope('Train') as scope:
    train_step =

Cuối cùng, định nghĩa hàm tính độ chính xác của thuật toán dự trên label thật $y\_correct$ và label dự đoán $y$ (của bất kì dataset nào, training/validation/test), tương tự như trong lab trước.

In [ ]:
with tf.name_scope("Accuracy"):
    correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_correct,1), name = "correct_prediction")
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

### Launch
Khởi tạo session và chạy thuật toán. Ở bước này, bạn nên copy code sang một file ".py". và chạy file này.

Dưới đây là một ví dụ cách chạy thuật toán và log kết quả biểu diễn trên TensorBoard.

In [ ]:
# Launch the model
sess = tf.InteractiveSession()
file_writer = tf.summary.FileWriter("DNN_1_hidden_layer_graphs", sess.graph)
# create a summary for our cost and accuracy
tf.summary.scalar("cost_summary", cost)
tf.summary.scalar("accuracy", accuracy)

# merge all summaries into a single operation which we can execute in a session 
summary_step = tf.summary.merge_all()

tf.global_variables_initializer().run()

for i in range(num_iterations):
    batch_xs, batch_ys = mnist.train.next_batch(batch_size)
    _, summary = sess.run([train_step, summary_step], feed_dict={x: batch_xs,y_correct: batch_ys})
    if i % 100 == 0:
        print("Accuracy validation: {}".format(accuracy.eval(feed_dict = {x: mnist.validation.images, y_correct: mnist.validation.labels})))
    # logging
    file_writer.add_summary(summary, i)
print("Accuracy: {}".format(accuracy.eval(feed_dict = {x: mnist.test.images, y_correct: mnist.test.labels})))

<b>Bài tập:</b> Hãy thay đổi giá trị của số lần lặp, tốc độ học, optimizer, cost function, activation function... và ghi chép lại kết quả thu được. Các số liệu này sẽ giúp ích cho các lab sau và final project. Hãy plot một trong những ảnh mà network này không phân loại đúng, one-hot vector mà network dự đoán cho ảnh này là gì?

## 3. Network với nhiều hidden layers


<b>Bài tập:</b> Hãy thêm hidden layers vào network ở trên, và theo dõi tốc độ học và độ chính xác của network mới.